In [0]:
import os
import pyspark
from pyspark import sql
from pyspark.sql import functions as f
from pyspark.sql import Window

In [0]:
dir_edw_data_parent = "/mnt/prod_edw/raw/cdc"
dir_brm_data_parent = "/mnt/prod_brm/raw/cdc"
dir_fs_data_parent = "/mnt/feature-store-prod-lab/d400_feature/d401_mobile_oa_consumer"
dir_fs_data_stg = "/mnt/feature-store-prod-lab/d200_staging/d299_src"
dir_fs_data_prm = '/mnt/feature-store-prod-lab/d300_primary/d301_mobile_oa_consumer/'
dir_global_meta = '/mnt/feature-store-dev/dev_users/dev_sc/d000_meta/d001_global_cycle_calendar'


In [0]:
df_raw_plan = spark.read.format('delta').load('/mnt/feature-store-prod-lab/d100_raw/d102_dwh_edw/raw_edw_product_rateplan/reporting_cycle_type=rolling cycle')

df_int_srvc = spark.read.format('delta').load('/mnt/feature-store-prod-lab/d200_intermediate/d201_mobile_oa_consumer/int_edw_service/reporting_cycle_type=rolling cycle')

# from feature store int layer 
df_int_ssc_srvc= spark.read.format('delta').load('/mnt/feature-store-prod-lab/d200_intermediate/d201_mobile_oa_consumer/int_ssc_service/')
df_int_ssc_billing = spark.read.format('delta').load('/mnt/feature-store-prod-lab/d200_intermediate/d201_mobile_oa_consumer/int_ssc_billing_account')
df_int_ssc_customer = spark.read.format('delta').load('/mnt/feature-store-prod-lab/d200_intermediate/d201_mobile_oa_consumer/int_ssc_customer')

df_int_d_billing_acct = spark.read.format('delta').load('/mnt/feature-store-prod-lab/d200_intermediate/d299_shared/int_d_billing_account')
df_int_d_service = spark.read.format('delta').load('/mnt/feature-store-prod-lab/d200_intermediate/d299_shared/int_d_service')

df_int_conn_deact = spark.read.format('delta').load('/mnt/feature-store-prod-lab/d200_intermediate/d299_shared/int_conn_deactivation_hist_mobile_oa')

In [0]:
display(
    df_int_conn_deact
    .filter(f.col('service_id') == '642102836626' )  
)

In [0]:
display(df_int_d_service.limit(10))

In [0]:
display(
    df_int_d_service
    .filter(f.col('fs_srvc_id') == '642102836626')
    .filter(f.col('billing_acct_src_id') == '1-WYX9CWX')
)

In [0]:
display(
    df_int_d_service
    .groupBy('srvc_type', 'srvc_status')
    .agg(f.count('*'))
)

In [0]:
display(
    df_int_ssc_srvc
    .filter(f.col('fs_srvc_id') == '642102836626')
    .filter(f.col('billing_acct_src_id') == '1-WYX9CWX')
) # scc does not include deactivation date 

In [0]:
display(
    df_int_ssc_billing
    .filter(
        f.col('billing_acct_num') == '473488431'
    )
)

In [0]:
display(
    df_int_ssc_srvc
    .filter(f.col('srvc_id') == '642102836626')
)

# 1-WYX9CWX

In [0]:
df_base_srvc_acct = (
        df_int_ssc_srvc
        .select("fs_acct_src_id")
        .distinct()
)

In [0]:
display(
    df_int_d_billing_acct
    .filter(f.col('billing_acct_num') == '473488431')   
)
# 1-WYX9CWX

In [0]:
 df_int_ssc_acct_curr_00 = (
                df_int_acct
                .filter(f.lower(f.col("billing_acct_status_desc")).isin(ls_param_acct_status))
                .filter(f.col("num_of_active_srvc_cnt") > 0)
                .filter(f.col("record_start_dttm") <= vt_param_ssc_end_date)
                # .filter(f.col("billing_acct_open_date") <= vt_param_ssc_end_date)
                .filter(f.col("record_end_dttm") >= vt_param_ssc_start_date)
                .withColumn("reporting_date", f.lit(vt_param_ssc_reporting_date))
                .withColumn("reporting_cycle_type", f.lit(vt_param_ssc_reporting_cycle_type))
                .withColumn("cycle_day_of_month_num", f.dayofmonth(f.col("reporting_date")))
                .withColumn(
                    "bill_month_shift_index"
                    , f.when(
                        f.col("cycle_day_of_month_num") <= f.col("bill_day_of_month_num")
                        , f.lit(-2)
                    ).otherwise(f.lit(-1))
                )
                .withColumn(
                    "bill_start_date"
                    , f.concat_ws(
                            "-"
                            , f.year(f.lit(vt_param_ssc_end_date))
                            , f.month(f.lit(vt_param_ssc_end_date))
                            , f.col("bill_day_of_month_num")
                    ).cast("date")
                )
            )

            df_int_ssc_acct_curr_01 = (
                df_int_ssc_acct_curr_00
                .filter(f.col("bill_month_shift_index") == -1)
                .withColumn("bill_start_date", f.add_months(f.col("bill_start_date"), -1))
            )

            df_int_ssc_acct_curr_02 = (
                df_int_ssc_acct_curr_00
                .filter(f.col("bill_month_shift_index") == -2)
                .withColumn("bill_start_date", f.add_months(f.col("bill_start_date"), -2))
            )

            df_output_curr = (
                df_int_ssc_acct_curr_01
                .union(df_int_ssc_acct_curr_02)
                .withColumn("bill_end_date", f.date_add(f.add_months(f.col("bill_start_date"), 1), -1))
                .withColumn("cal_start_date", f.lit(vt_param_ssc_start_date))
                .withColumn("cal_end_date", f.lit(vt_param_ssc_end_date))
                .withColumn(
                    "index"
                    , f.row_number().over(
                        Window
                        .partitionBy("fs_acct_src_id", "fs_cust_id", "reporting_date")
                        .orderBy(f.desc("record_end_dttm"), f.desc("record_start_dttm"))
                    )
                )
                .filter(f.col("index") == 1)
                .drop("index", "cycle_day_of_month_num", "bill_month_shift_index")
                # narrow down to Mobile OA consumer
                .join(df_base_srvc_acct, ['fs_acct_src_id'], "inner")
                .withColumn("data_update_date", f.current_date())
                .withColumn("data_update_dttm", f.current_timestamp())
            )

In [0]:
display(
    df_int_ssc_billing
    .filter(
        f.col('billing_acct_num') =='473488431'
    )
)

In [0]:
display(
    df_int_ssc_service
    .filter(f.col('srvc_id') == '642102836626')    
    .filter(f.col(''))
)